# CMSC848F Assignment 1
## Name: Hritvik Choudhari
## UID: 119208793

### Importing necessary libraries and dependencies

In [ ]:
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith(("1.13.", "2.0.")) and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

In [ ]:
!wget https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/docs/tutorials/utils/plot_image_grid.py

In [3]:
import sys
sys.path.append("/content/drive/MyDrive/assignment1-main/")

In [ ]:
!pip install PyMCubes

# **0.1 Rendering your first mesh**

In [13]:
from starter.utils import get_device, get_mesh_renderer, load_cow_mesh
import pytorch3d, torch
import matplotlib.pyplot as plt

# Get the device (CPU or GPU)
device = get_device()

# Load cow mesh vertices and faces
vertices, faces = load_cow_mesh('/content/drive/MyDrive/assignment1-main/data/cow.obj')

def section0_1(vertices, faces, output_path, color, image_size):
    # Get the renderer.
    renderer = get_mesh_renderer(image_size=image_size)

    # Convert vertices and faces to PyTorch tensors
    vertices = torch.tensor(vertices, dtype=torch.float32)
    faces = torch.tensor(faces, dtype=torch.int32)

    # Add batch dimension (1) to vertices and faces
    vertices = vertices.unsqueeze(0)  # (N_v, 3) -> (1, N_v, 3)
    faces = faces.unsqueeze(0)  # (N_f, 3) -> (1, N_f, 3)

    # Create a texture using the specified color
    textures = torch.ones_like(vertices)  # (1, N_v, 3)
    textures = textures * torch.tensor(color)  # (1, N_v, 3)

    # Create a Meshes object with vertices, faces, and textures
    mesh = pytorch3d.structures.Meshes(
        verts=vertices,
        faces=faces,
        textures=pytorch3d.renderer.TexturesVertex(textures),
    )
    mesh = mesh.to(device)

    # Define camera parameters (position, orientation, and field of view)
    cameras = pytorch3d.renderer.FoVPerspectiveCameras(
        R=torch.eye(3).unsqueeze(0),  # Identity rotation matrix
        T=torch.tensor([[0, 0, 3]]),  # Translation along the z-axis
        fov=90,  # Field of view (in degrees)
        device=device
    )

    # Define lighting parameters (position of point light source)
    lights = pytorch3d.renderer.PointLights(location=[[-1, -1, -3]], device=device)

    # Render the mesh using the specified camera and lights
    rend = renderer(mesh, cameras=cameras, lights=lights)

    # Extract the RGB channels and convert to a NumPy array
    rend = rend.cpu().numpy()[0, ..., :3]  # (B, H, W, 4) -> (H, W, 3)

    # Save the rendered image as a JPEG file
    plt.imsave(output_path, rend)

# Call the function to render and save the image
section0_1(vertices, faces, '/content/drive/MyDrive/assignment1-main/section0_1_render.jpg', [0.3, 0.9, 0.5], 512)


<ipython-input-13-1f9d7ea423a0>:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  vertices = torch.tensor(vertices, dtype=torch.float32)
<ipython-input-13-1f9d7ea423a0>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  faces = torch.tensor(faces, dtype=torch.int32)


# 1. Practicing with Cameras

## 1.1. 360-degree Renders

In [15]:
import torch
import pytorch3d
import matplotlib.pyplot as plt
import imageio
from starter.utils import get_device, get_mesh_renderer, load_cow_mesh

# Get the device (CPU or GPU)
device = get_device()

# Load cow mesh vertices and faces
vertices, faces = load_cow_mesh('/content/drive/MyDrive/assignment1-main/data/cow.obj')

# Add batch dimension to vertices and faces
vertices = vertices.unsqueeze(0)  # (N_v, 3) -> (1, N_v, 3)
faces = faces.unsqueeze(0)  # (N_f, 3) -> (1, N_f, 3)

# Create a texture for the mesh
textures = torch.ones_like(vertices)  # (1, N_v, 3)
textures = textures * torch.tensor([0.5, 0.2, 0.1])  # (1, N_v, 3)

# Create a Meshes object with vertices, faces, and textures
mesh = pytorch3d.structures.Meshes(
    verts=vertices,
    faces=faces,
    textures=pytorch3d.renderer.TexturesVertex(textures),
)
mesh = mesh.to(device)

def section1_1(num_frames, mesh, image_size):
    # Define the number of frames for the 360-degree rotation.
    num_frames = num_frames
    renderer = get_mesh_renderer(image_size=image_size)

    # Generate a list of camera positions to cover a full rotation.
    camera_rotation, camera_translation = pytorch3d.renderer.look_at_view_transform(
        dist=-4.0, elev=0, azim=torch.linspace(0, 360, num_frames), device=device
    )

    # Create a list to store the rendered images.
    rendered_images = []

    # Render images for each camera position.
    for camera_pose in zip(camera_rotation, camera_translation):
        # Update the camera position.
        R = camera_pose[0].unsqueeze(0)
        T = camera_pose[1].unsqueeze(0)
        cameras = pytorch3d.renderer.FoVPerspectiveCameras(
            R=R,
            T=T,
            device=device,
        )

        # Render the image.
        rendered_image = renderer(mesh, cameras=cameras)
        if rendered_image.shape[3] == 4:
            rendered_image = rendered_image[:, :, :, :3]  # Keep only RGB channels

        # Convert the rendered image to a numpy array and add it to the list.
        rendered_image_rgb = (rendered_image.squeeze() * 255).byte().cpu().numpy()
        rendered_images.append(rendered_image_rgb)

    return rendered_images

# Save the rendered images as frames for the gif.
image_list = section1_1(36, mesh, 512)
output_path = "/content/drive/MyDrive/assignment1-main/section1_1.gif"
imageio.mimsave(output_path, image_list, duration=50, loop=0)  # Adjust the duration as needed


## 1.2 Re-creating the Dolly Zoom

In [17]:
import argparse
import imageio
import numpy as np
import pytorch3d
import torch
from PIL import Image, ImageDraw
from tqdm.auto import tqdm
from starter.utils import get_device, get_mesh_renderer

def section1_2(output_file,
    image_size=256,
    num_frames=10,
    duration=100,
    device=None
):
    # Check if a specific device is specified, otherwise get the default device.
    if device is None:
        device = get_device()

    # Load the cow mesh.
    mesh = pytorch3d.io.load_objs_as_meshes(["/content/drive/MyDrive/assignment1-main/data/cow_on_plane.obj"])
    mesh = mesh.to(device)

    # Create a mesh renderer.
    renderer = get_mesh_renderer(image_size=image_size, device=device)

    # Define point lights.
    lights = pytorch3d.renderer.PointLights(location=[[0.0, 0.0, -3.0]], device=device)

    # Create a sequence of field of view (FOV) values for zoom effect.
    fovs = torch.linspace(0, 125, num_frames)

    # List to store rendered images.
    renders = []

    # Iterate over FOV values and render frames.
    for fov in tqdm(fovs):
        # Calculate the distance based on FOV to achieve the dolly zoom effect.
        distance = 2.5 / np.tan(np.radians(fov / 2))

        # Set the camera translation to create dolly zoom effect.
        T = [[0, 0, distance]]

        # Create perspective cameras with the specified FOV and translation.
        cameras = pytorch3d.renderer.FoVPerspectiveCameras(fov=fov, T=T, device=device)

        # Render the image.
        rend = renderer(mesh, cameras=cameras, lights=lights)

        # Extract the RGB channels.
        rend = rend[0, ..., :3].cpu().numpy()
        renders.append(rend)

    # Create a list to store final images with text annotations.
    images = []

    # Iterate over rendered frames.
    for i, r in enumerate(renders):
        # Convert the numpy array to PIL Image.
        image = Image.fromarray((r * 255).astype(np.uint8))

        # Create a drawing context to add text annotations.
        draw = ImageDraw.Draw(image)

        # Add FOV information as text.
        draw.text((20, 20), f"fov: {fovs[i]:.2f}", fill=(255, 0, 0))

        # Append the annotated image to the list.
        images.append(np.array(image))

    # Save the list of images as a GIF.
    imageio.mimsave(output_file, images, duration=duration, loop=0)

# Call the dolly_zoom function with the specified parameters.
section1_2(image_size=256,
        num_frames=30,
        duration=150,
        output_file='/content/drive/MyDrive/assignment1-main/section1_2.gif'
    )


  0%|          | 0/30 [00:00<?, ?it/s]

# 2. Practicing with Meshes

## 2.1 Constructing a Tetrahedron

In [19]:
from starter.utils import get_device, get_mesh_renderer, load_cow_mesh
import pytorch3d, torch
import matplotlib.pyplot as plt

# Get the device for computation (e.g., CUDA GPU or CPU).
device = get_device()

# Define the vertices of the mesh.
vertices = torch.tensor([[
    [0.0, 0.0, 0.0],   # Vertex 1
    [1.0, 0.0, 0.0],   # Vertex 2
    [0.5, 1.0, 0.0],   # Vertex 3
    [0.5, 0.5, 1.0],   # Vertex 4
]], dtype=torch.float32)

# Define the faces of the mesh.
faces = torch.tensor([[
    [0, 1, 2],         # Face 1 (vertices 0, 1, 2)
    [0, 1, 3],         # Face 2 (vertices 0, 1, 3)
    [0, 2, 3],         # Face 3 (vertices 0, 2, 3)
    [1, 2, 3],         # Face 4 (vertices 1, 2, 3)
]], dtype=torch.int32)

# Define the textures for the mesh.
textures = torch.ones_like(vertices)  # (1, N_v, 3)
textures = textures * torch.tensor([0.1, 0.7, 0.9])  # (1, N_v, 3)

# Create a Meshes object with vertices, faces, and textures.
mesh = pytorch3d.structures.Meshes(
    verts=vertices,
    faces=faces,
    textures=pytorch3d.renderer.TexturesVertex(textures),
)

# Move the mesh to the specified device (e.g., GPU).
mesh = mesh.to(device)

# Define a function to render images from different camera angles.
def section2_1(mesh, num_frames, image_size):
    num_frames = num_frames
    renderer = get_mesh_renderer(image_size=image_size)

    # Generate a list of camera positions to cover a full rotation.
    camera_rotation, camera_translation = pytorch3d.renderer.look_at_view_transform(
        dist= -4.0, elev=0, azim=torch.linspace(0, 360, num_frames), device=device
    )

    # Create a list to store the rendered images.
    rendered_images = []

    # Render images for each camera position.
    for camera_pose in zip(camera_rotation, camera_translation):
        # Update the camera position.
        R = camera_pose[0].unsqueeze(0)
        T = camera_pose[1].unsqueeze(0)
        cameras = pytorch3d.renderer.FoVPerspectiveCameras(
            R=R,
            T=T,
            device=device,
        )

        # Render the image.
        rendered_image = renderer(mesh, cameras=cameras)

        # Keep only the RGB channels if there's an alpha channel.
        if rendered_image.shape[3] == 4:
            rendered_image = rendered_image[:, :, :, :3]

        # Convert the rendered image to a numpy array and add it to the list.
        rendered_image_rgb = (rendered_image.squeeze() * 255).byte().cpu().numpy()
        rendered_images.append(rendered_image_rgb)

    return rendered_images

# Save the rendered images as frames for the gif.
image_list = section2_1(mesh, num_frames=36, image_size=512)
output_path = "/content/drive/MyDrive/assignment1-main/section2_1.gif"
imageio.mimsave(output_path, image_list, duration=50, loop=0)


## 2.2 Constructing a Cube

In [20]:
from starter.utils import get_device, get_mesh_renderer, load_cow_mesh
import pytorch3d, torch
import matplotlib.pyplot as plt

# Get the device for computation (e.g., CUDA GPU or CPU).
device = get_device()

# Define the vertices of a cube.
vertices = torch.tensor([[
    [-1, -1, -1],  # Vertex 1
    [1, -1, -1],   # Vertex 2
    [1, 1, -1],    # Vertex 3
    [-1, 1, -1],   # Vertex 4
    [-1, -1, 1],   # Vertex 5
    [1, -1, 1],    # Vertex 6
    [1, 1, 1],     # Vertex 7
    [-1, 1, 1],    # Vertex 8
]], dtype=torch.float32)

# Define the faces of the cube using two sets of triangles.
faces = torch.tensor([[
    [0, 1, 2], [0, 2, 3],  # Bottom face (vertices 0, 1, 2 and 0, 2, 3)
    [4, 5, 6], [4, 6, 7],  # Top face (vertices 4, 5, 6 and 4, 6, 7)
    [0, 1, 5], [0, 5, 4],  # Front face (vertices 0, 1, 5 and 0, 5, 4)
    [2, 3, 7], [2, 7, 6],  # Back face (vertices 2, 3, 7 and 2, 7, 6)
    [1, 2, 6], [1, 6, 5],  # Right face (vertices 1, 2, 6 and 1, 6, 5)
    [0, 3, 7], [0, 7, 4],  # Left face (vertices 0, 3, 7 and 0, 7, 4)
]], dtype=torch.int32)

# Define the textures for the cube.
textures = torch.ones_like(vertices)  # (1, N_v, 3)
textures = textures * torch.tensor([0.1, 0.7, 0.9])  # (1, N_v, 3)

# Create a Meshes object with vertices, faces, and textures.
mesh = pytorch3d.structures.Meshes(
    verts=vertices,
    faces=faces,
    textures=pytorch3d.renderer.TexturesVertex(textures),
)

# Move the mesh to the specified device (e.g., GPU).
mesh = mesh.to(device)

# Define a function to render images from different camera angles.
def section2_2(mesh, num_frames, image_size):
    num_frames = num_frames
    renderer = get_mesh_renderer(image_size=image_size)

    # Generate a list of camera positions to cover a full rotation.
    camera_rotation, camera_translation = pytorch3d.renderer.look_at_view_transform(
        dist=-5.0, elev=0, azim=torch.linspace(0, 360, num_frames), device=device
    )

    # Create a list to store the rendered images.
    rendered_images = []

    # Render images for each camera position.
    for camera_pose in zip(camera_rotation, camera_translation):
        # Update the camera position.
        R = camera_pose[0].unsqueeze(0)
        T = camera_pose[1].unsqueeze(0)
        cameras = pytorch3d.renderer.FoVPerspectiveCameras(
            R=R,
            T=T,
            device=device,
        )

        # Render the image.
        rendered_image = renderer(mesh, cameras=cameras)

        # Keep only the RGB channels if there's an alpha channel.
        if rendered_image.shape[3] == 4:
            rendered_image = rendered_image[:, :, :, :3]

        # Convert the rendered image to a numpy array and add it to the list.
        rendered_image_rgb = (rendered_image.squeeze() * 255).byte().cpu().numpy()
        rendered_images.append(rendered_image_rgb)

    return rendered_images

# Save the rendered images as frames for the gif.
image_list = section2_2(mesh, num_frames=36, image_size=512)
output_path = "/content/drive/MyDrive/assignment1-main/section2_2.gif"
imageio.mimsave(output_path, image_list, duration=50, loop=0)


# 3. Re-texturing a mesh

In [22]:
import torch
import pytorch3d
import matplotlib.pyplot as plt
import imageio
from starter.utils import get_device, get_mesh_renderer, load_cow_mesh

# Get the device for computation (e.g., CUDA GPU or CPU).
device = get_device()

# Load the cow mesh vertices and faces from a file.
vertices, faces = load_cow_mesh('/content/drive/MyDrive/assignment1-main/data/cow.obj')

# Move vertices and faces to the specified device and add batch dimensions.
vertices = vertices.unsqueeze(0).to(device)  # (N_v, 3) -> (1, N_v, 3)
faces = faces.unsqueeze(0).to(device)  # (N_f, 3) -> (1, N_f, 3)

# Calculate the minimum and maximum z-coordinates in the mesh.
z_min = vertices[:, :, 2].min()
z_max = vertices[:, :, 2].max()

# Define two colors for the front and back faces.
color1 = torch.tensor([0, 1, 1], dtype=torch.float32, device=device)
color2 = torch.tensor([1, 0.2, 0.5], dtype=torch.float32, device=device)

# Compute interpolated colors based on vertex depth.
alphas = (vertices[:, :, 2] - z_min) / (z_max - z_min)
colors = (alphas.unsqueeze(-1) * color2 + (1 - alphas.unsqueeze(-1)) * color1)

# Create a Meshes object with vertices, faces, and vertex colors.
mesh = pytorch3d.structures.Meshes(
    verts=vertices,
    faces=faces,
    textures=pytorch3d.renderer.TexturesVertex(colors),
)
mesh = mesh.to(device)

# Define a function to render images from different camera angles.
def section3(num_frames, mesh, image_size):
    # Define the number of frames for the 360-degree rotation.
    num_frames = num_frames
    renderer = get_mesh_renderer(image_size=image_size)

    # Generate a list of camera positions to cover a full rotation.
    camera_rotation, camera_translation = pytorch3d.renderer.look_at_view_transform(
        dist= -4.0, elev=0, azim=torch.linspace(0, 360, num_frames), device=device
    )

    # Create a list to store the rendered images.
    rendered_images = []

    # Render images for each camera position.
    for camera_pose in zip(camera_rotation, camera_translation):
        # Update the camera position.
        R = camera_pose[0].unsqueeze(0)
        T = camera_pose[1].unsqueeze(0)
        cameras = pytorch3d.renderer.FoVPerspectiveCameras(
            R=R,
            T=T,
            device=device,
        )

        # Render the image.
        rendered_image = renderer(mesh, cameras=cameras)

        # Keep only the RGB channels if there's an alpha channel.
        if rendered_image.shape[3] == 4:
            rendered_image = rendered_image[:, :, :, :3]

        # Convert the rendered image to a numpy array and add it to the list.
        rendered_image_rgb = (rendered_image.squeeze() * 255).byte().cpu().numpy()
        rendered_images.append(rendered_image_rgb)

    return rendered_images

# Save the rendered images as frames for the gif.
image_list = section3(36, mesh, 512)
output_path = "/content/drive/MyDrive/assignment1-main/section3.gif"
imageio.mimsave(output_path, image_list, duration=100, loop=0)


# 4. Camera Transformations

In [24]:
import matplotlib.pyplot as plt
import pytorch3d
import torch
import numpy as np

from starter.utils import get_device, get_mesh_renderer

# Function to render the cow with different poses.
def render_cow(R_relative, T_relative, cow_path="data/cow_with_axis.obj", image_size=256, device=None):
    if device is None:
        device = get_device()

    # Load the cow mesh as Meshes object.
    meshes = pytorch3d.io.load_objs_as_meshes([cow_path]).to(device)

    # Calculate the updated rotation and translation matrices for the camera.
    R = R_relative @ torch.tensor([[1.0, 0, 0], [0, 1, 0], [0, 0, 1]])
    T = R_relative @ torch.tensor([0.0, 0, 3]) + T_relative

    # Create a mesh renderer and define cameras and lights.
    renderer = get_mesh_renderer(image_size=image_size)
    cameras = pytorch3d.renderer.FoVPerspectiveCameras(
        R=R.t().unsqueeze(0), T=T.unsqueeze(0), device=device,
    )
    lights = pytorch3d.renderer.PointLights(location=[[0, 0.0, -3.0]], device=device)

    # Render the image.
    rend = renderer(meshes, cameras=cameras, lights=lights)

    # Return the rendered image.
    return rend[0, ..., :3].cpu().numpy()

# Define different poses for the cow.
pose1 = (
    torch.tensor([
        [torch.cos(torch.tensor(90.0).to(torch.float32) * (torch.pi / 180)), -torch.sin(torch.tensor(90.0).to(torch.float32) * (torch.pi / 180)), 0],
        [torch.sin(torch.tensor(90.0).to(torch.float32) * (torch.pi / 180)), torch.cos(torch.tensor(90.0).to(torch.float32) * (torch.pi / 180)), 0],
        [0, 0, 1]
    ]).to(torch.float32),
    [0, 0, 0]
)

pose2 = (
    torch.tensor([[1, 0, 0], [0, 1, 0], [0, 0, 1]]).to(torch.float32),
    [0, 0, 3]
)

pose3 = (
    torch.tensor([
        [torch.cos(torch.tensor(10.0).to(torch.float32) * (torch.pi / 180)), 0, -torch.sin(torch.tensor(10.0).to(torch.float32) * (torch.pi / 180))],
        [0, 1, 0],
        [torch.sin(torch.tensor(10.0).to(torch.float32) * (torch.pi / 180)), 0, torch.cos(torch.tensor(10.0).to(torch.float32) * (torch.pi / 180))]
    ]).to(torch.float32),
    [0.75, -0.25, -0.45]
)

pose4 = (
    torch.tensor([
        [torch.cos(torch.tensor(90.0).to(torch.float32) * (torch.pi / 180)), 0, -torch.sin(torch.tensor(90.0).to(torch.float32) * (torch.pi / 180))],
        [0, 1, 0],
        [torch.sin(torch.tensor(90.0).to(torch.float32) * (torch.pi / 180)), 0, torch.cos(torch.tensor(90.0).to(torch.float32) * (torch.pi / 180))]
    ]).to(torch.float32),
    [3, 0, 3]
)

# Create a list of poses.
poses = [pose1, pose2, pose3, pose4]

# Render and save images for each pose.
for i, val in enumerate(poses):
    plt.imsave("/content/drive/MyDrive/assignment1-main/section4_{}.jpg".format(i+1), render_cow(cow_path="/content/drive/MyDrive/assignment1-main/data/cow_with_axis.obj", image_size=256, R_relative=val[0], T_relative=torch.tensor(val[1])))


# 5. Rendering Generic 3D Representations

## 5.1 Rendering Point Clouds from RGB-D Images

In [30]:
from starter.utils import get_points_renderer, get_device, unproject_depth_image
from starter.render_generic import load_rgbd_data
import pytorch3d
import imageio

# Get the device (CPU or GPU) for PyTorch operations.
device = get_device()

# Load the RGBD data from a pickle file.
data = load_rgbd_data(path="/content/drive/MyDrive/assignment1-main/data/rgbd_data.pkl")

# Separate data for the first and second images.
data_img1 = [data['rgb1'], data['mask1'], data['depth1'], data['cameras1']]
data_img2 = [data['rgb2'], data['mask2'], data['depth2'], data['cameras2']]

# Unproject the depth image to get point clouds and RGB values for both images.
pc1, rgb1 = unproject_depth_image(torch.Tensor(data_img1[0]), torch.Tensor(data_img1[1]), torch.Tensor(data_img1[2]), data_img1[3])
pc2, rgb2 = unproject_depth_image(torch.Tensor(data_img2[0]), torch.Tensor(data_img2[1]), torch.Tensor(data_img2[2]), data_img2[3])

# Combine the point clouds and RGB values from both images.
combined_points = torch.cat([pc1, pc2], dim=0)
combined_features = torch.cat([rgb1, rgb2], dim=0)

# Define a function to render the combined point cloud with rotation.
def section5_1(pc, rgb, rotation_matrix, device):
    # Prepare the points and RGB data and apply the rotation matrix.
    points = pc.unsqueeze(0)  # 1 x N x 3
    rgb = rgb.unsqueeze(0)    # 1 x N x 3
    points = torch.matmul(points, rotation_matrix.T)

    # Create a Pointclouds object.
    point_cloud = pytorch3d.structures.Pointclouds(
        points=points, features=rgb
    ).to(device)

    # Get the renderer for points.
    points_renderer = get_points_renderer(
        image_size=256,
        radius=0.01,
    )

    # Generate camera positions for rendering.
    camera_rotation, camera_translation = pytorch3d.renderer.look_at_view_transform(
        dist=7.0, elev=0.0, azim=torch.linspace(0, 360, 36), device=device
    )

    # Create a list to store the rendered images.
    rendered_images = []

    # Render images for each camera position.
    for camera_pose in zip(camera_rotation, camera_translation):
        # Update the camera position.
        R = camera_pose[0].unsqueeze(0)
        T = camera_pose[1].unsqueeze(0)
        cameras = pytorch3d.renderer.FoVPerspectiveCameras(
            R=R,
            T=T,
            device=device,
        )
        rendered_image = points_renderer(point_cloud, cameras=cameras)

        # Keep only RGB channels (remove alpha if exists).
        if rendered_image.shape[3] == 4:
            rendered_image = rendered_image[:, :, :, :3]

        # Convert the rendered image to a numpy array and add it to the list.
        rendered_image_rgb = (rendered_image.squeeze() * 255).byte().cpu().numpy()
        rendered_images.append(rendered_image_rgb)

    return rendered_images

# Define a rotation matrix for the point cloud.
rotation_matrix = torch.tensor([
    [-1, 0, 0],
    [0, -1, 0],
    [0, 0, 1]
], dtype=torch.float32)

# Render the point cloud with rotation and save it as a GIF.
image_list = section5_1(pc2, rgb2, rotation_matrix, device)
output_path = "/content/drive/MyDrive/assignment1-main/section5_1_2.gif"
imageio.mimsave(output_path, image_list, duration=100, loop=0)


## 5.2 Parametric Functions

In [5]:
import matplotlib.pyplot as plt
import pytorch3d
import torch
from torch import cos, sin
import numpy as np
import imageio
from starter.utils import get_points_renderer, get_device

def section5_2(azimuth, R=0.4, r=0.2, image_size=256, num_samples= 150, device=None):
    """
    Renders a torus using parametric sampling. Samples num_samples ** 2 points.
    """

    if device is None:
        device = get_device()

    # Generate phi and theta values for sampling.
    phi_vals = torch.linspace(0, 2 * np.pi, num_samples)
    theta_vals = torch.linspace(0, 2 * np.pi, num_samples)

    # Create a grid of phi and theta values.
    phi_mesh, theta_mesh = torch.meshgrid(phi_vals, theta_vals)

    # Calculate x, y, and z coordinates of points on the torus.
    x = (R + r * cos(phi_mesh)) * cos(theta_mesh)
    y = (R + r * cos(phi_mesh)) * sin(theta_mesh)
    z = r * sin(phi_mesh)

    # Stack the coordinates to create the point cloud.
    points = torch.stack((x.flatten(), y.flatten(), z.flatten()), dim=1)

    # Normalize the color based on the point cloud coordinates.
    color = (points - points.min()) / (points.max() - points.min())

    # Create a Pointclouds object with points and colors.
    torus_pc = pytorch3d.structures.Pointclouds(
        points=[points], features=[color],
    ).to(device)

    # Define the view transformation using azimuth angle.
    R, T = pytorch3d.renderer.look_at_view_transform(2, 3, azimuth, degrees=False)
    cameras = pytorch3d.renderer.FoVPerspectiveCameras(R=R, T=T, device=device)

    # Get the points renderer and render the torus.
    renderer = get_points_renderer(image_size=image_size, device=device)
    rend = renderer(torus_pc, cameras=cameras)
    rend = rend[0, ..., :3]  # (B, H, W, 4) -> (H, W, 3)

    return (rend.squeeze() * 255).byte().cpu().numpy()

# Define a range of azimuth angles to create a 360-degree view.
azimuth = torch.linspace(0, 2 * np.pi, 36)
torus_views = []

# Render the torus from different viewpoints and collect the images.
for i in azimuth:
    view = section5_2(i)
    torus_views.append(view)

# Generate a GIF for the 360-degree view.
imageio.mimsave('/content/drive/MyDrive/assignment1-main/section5_2.gif', torus_views, duration=70, loop=0)


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


## 5.3 Implicit Surfaces

In [6]:
import matplotlib.pyplot as plt
import pytorch3d
import torch
import numpy as np
import imageio
import mcubes
from starter.utils import get_mesh_renderer, get_device

def section5_3(azimuth, R=0.4, r=0.2, image_size=256, num_samples= 200, device=None):
    """
    Renders a torus using parametric sampling. Samples num_samples ** 2 points.

    Parameters:
    - azimuth (float): The azimuth angle for the viewpoint.
    - R (float): Major radius of the torus.
    - r (float): Minor radius of the torus.
    - image_size (int): Size of the rendered image.
    - num_samples (int): Number of samples for parametric sampling.
    - device (str): Device (e.g., 'cuda' or 'cpu').

    Returns:
    - numpy.ndarray: Rendered image as a numpy array.
    """

    if device is None:
        device = get_device()

    # Define voxel grid parameters.
    voxel_size = 64
    min_value = -1.1
    max_value = 1.1

    # Create a 3D grid for voxels.
    X, Y, Z = torch.meshgrid([torch.linspace(min_value, max_value, voxel_size)] * 3)

    # Define the implicit function for the torus using voxels.
    voxels = (torch.sqrt(X ** 2 + Y ** 2) - R) ** 2 + Z ** 2 - r ** 2

    # Extract vertices and faces using Marching Cubes.
    vertices, faces = mcubes.marching_cubes(mcubes.smooth(voxels), isovalue=0)

    # Convert vertices and faces to tensors.
    vertices = torch.tensor(vertices).float()
    faces = torch.tensor(faces.astype(int))

    # Normalize vertex coordinates.
    vertices = (vertices / voxel_size) * (max_value - min_value) + min_value

    # Define textures based on vertex positions.
    textures = (vertices - vertices.min()) / (vertices.max() - vertices.min())
    textures = pytorch3d.renderer.TexturesVertex(textures.unsqueeze(0))

    # Create a PyTorch3D Mesh representing the torus.
    torus_mesh = pytorch3d.structures.Meshes(
        [vertices],
        [faces],
        textures=textures
    ).to(device)

    # Define the view transformation using azimuth angle.
    R, T = pytorch3d.renderer.look_at_view_transform(2, 3, azimuth, degrees=False)
    cameras = pytorch3d.renderer.FoVPerspectiveCameras(R=R, T=T, device=device)

    # Get the mesh renderer and render the torus.
    renderer = get_mesh_renderer(image_size=image_size, device=device)
    rend = renderer(torus_mesh, cameras=cameras)

    # Extract the RGB channels from the rendered image.
    rend = rend[0, ..., :3]  # (B, H, W, 4) -> (H, W, 3)

    return (rend.squeeze() * 255).byte().cpu().numpy()

# Define a range of azimuth angles to create a 360-degree view.
azimuth = torch.linspace(0, 2 * np.pi, 36)
torus_views = []

# Render the torus from different viewpoints and collect the images.
for i in azimuth:
    view = section5_3(i)
    torus_views.append(view)

# Generate a GIF for the 360-degree view.
imageio.mimsave('/content/drive/MyDrive/assignment1-main/section5_3.gif', torus_views, duration=70, loop=0)
